# Monte Carlo Random Walk Simulation

### Libraries

In [2]:
library("PopED")
library("sets")
library("rlist")
library("data.table")
library("dplyr")
options(warn=-1)

## Approach

1) The function `random_walk` takes the starting position, ending position and a matrix of 4 x 4 as input.
2) While (the condition is TRUE):
    - define a set of 4 directions in which the person can move
    - update the set of directions based on edges of a 4 x 4 matrix
    - update the set of directions based on if the adjacent boxes have already been explored
    - select a random direction from the list of directions and move in that way and update the current coordinate.
    - add the direction to the path 
    - if a dead end is reached then return OR if the person reaches 4,4 position then also return.
    

In [3]:
# Function Input: Takes a direction coodinate as input
# Function Output: Returns a string direction
# Example: Input: (0,1), Output: RIGHT

direction_string_function <- function(direction) {
    
 # Coordinates are like (y,x) in a matrix
    
    
   if (direction[[1]][1] == 0 & direction[[1]][2] == 1) {
       
       return("RIGHT")
  }
    else if (direction[[1]][1] == 0 & direction[[1]][2] == -1) {
       
       return("LEFT")
  }
    else if (direction[[1]][1] == 1 & direction[[1]][2] == 0) {
       
       return("DOWN")
  }
    else if (direction[[1]][1] == -1 & direction[[1]][2] == 0) {
       
       return("UP")
  }

}

In [4]:
# Function Input: Takes the starting position (current_pos), tracking matrix of 4x4 and ending position (4,4)
# Function Output: Returns the path of a random walk, end result type and matrix sum if (4,4) end point is reached.
# Example: Input: random_walk(c(4,4), c(1,1), zeros(4,4)), Output: RIGHT LEFT DOWN UP RIGHT, 0 OR 1, 13

random_walk <- function(end_pos, current_pos, tracking) {
    
    path <- c('PATH')
    counter <- TRUE
    while (counter) {
        
        # y is the first coordinate.      
        y <- current_pos[1]
        x <- current_pos[2]
        
        directions <- list(right = c(0,1), left = c(0,-1), down = c(1,0), up = c(-1,0))
        if (y == 1) {
            # cannot move UP 
            directions <- within(directions, rm(up)) 
            
        }
                
        if (x == 1) {
            # cannot move LEFT
            directions <- within(directions, rm(left)) 

            
        }
            
        if (y == 4) {
            # cannot move down
            directions <- within(directions, rm(down)) 

    
        }
        
        if (x == 4) {
            # cannot move right
            directions <- within(directions, rm(right)) 
        }
        
    
        # For all pair of directions in the list check whether you have visited all
        # remaining these directions and eliminate all options which lead you to a revisited cell
        remove_labels <- list()
        for (i in 1:length(directions)) {
        
          temp_pos <- c(0,0)
          y_cor = directions[[i]][1]
          x_cor = directions[[i]][2]
          temp_pos[1] <-  current_pos[1] +  y_cor
            # x
          temp_pos[2] <- current_pos[2] + x_cor
            
            # if the coordinate has already been visited then extract the label of the direction and store in remove_labels
            
          if (tracking[temp_pos[1],temp_pos[2]] == 1) {
              
              dir_label <- direction_string_function(directions[i])
              dir_label <- stringr::str_to_lower(dir_label)
              remove_labels <- list.append(remove_labels,dir_label)
               
              }
        

          }
        
        # Remove directions in which you cannot move
         for (i in remove_labels) {
            directions[[i]] <- NULL
         }
        
         
        # Check the remaining length of the list. If it is 0 then no directions to move in hence dead end. 
        if (length(directions) == 0) {
            # print('dead end reached')
            counter <- FALSE
            result = list("path" = path, "end_result" = 0)
            return(result)
        }

        # Update coordinates in the chosen_direction
        n = length(directions)
        v <- rep(1, n)
        chosen_direction <- sample (directions, size = 1, replace=T, prob = v * 1/n)
        
        # y
        current_pos[1] <- current_pos[1] + chosen_direction[[1]][1]
        # x
        current_pos[2] <- current_pos[2] + chosen_direction[[1]][2]  
        
        # Place 1 in the boxes which have been visited
        tracking[current_pos[1],current_pos[2]] <- 1
#         print(tracking)
        
        
        # Add string to path
        direction_string <- direction_string_function(chosen_direction)
        path <- append(path, direction_string)
        
        # Location is reached when 4,4 
        if (current_pos[1] == 4 & current_pos[2] == 4) {
            
            counter <- FALSE
            result = list("path" = path, "end_result" = 1, "matrix_sum" = sum(tracking))
            return(result)
        }


    } # While
}

In [5]:
# Inputs to the random_walk function
tracking <- zeros(4, 4)
end_pos <- c(4,4)
current_pos <- c(1,1)

### Monte Carlo Simulation

- Run 10000 walkers
- If a walker reaches an end point by visiting all the boxes then return it's path and count the number of unique occurences.

In [6]:
matrix_sum_counter <- 1
dt <- data.table()
for (i in 1:10000) {
    results <- random_walk(end_pos, current_pos, tracking)
    
    # end point reached
    if (results$end_result == 1) {
        
        if (results$matrix_sum == 16) {
            
            index <- matrix_sum_counter
            # place each path as a column in the datatable
            dt[, paste0("path_",index):=results$path[-c(1)]]
            matrix_sum_counter = matrix_sum_counter + 1
        }
    }
    
}


In [7]:
# converting datatable to dataframe and transposing it to find the unique solutions
df <- as.data.frame(dt)
# 16 columns represent 16 steps to reach end (4,4)
df_transpose <- transpose(df)
final_df <- transpose(distinct(df_transpose))

### Final paths

In [8]:
# Remove duplicate paths to find the number of unique paths
cat('Number of unique solutions: ',nrow(distinct(df_transpose)))

Number of unique solutions:  8

In [9]:
# Rename the columns
final_df %>% 
  rename(
    path_1 = V1,
    path_2 =  V2,
    path_3 = V3,
    path_4 = V4,
    path_5 = V5,
    path_6 = V6,
    path_7 = V7,
    path_8 = V8
    )

path_1,path_2,path_3,path_4,path_5,path_6,path_7,path_8
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
RIGHT,DOWN,RIGHT,DOWN,DOWN,RIGHT,DOWN,RIGHT
LEFT,UP,LEFT,UP,UP,LEFT,UP,LEFT
DOWN,RIGHT,DOWN,RIGHT,RIGHT,DOWN,RIGHT,DOWN
RIGHT,RIGHT,DOWN,DOWN,DOWN,DOWN,RIGHT,RIGHT
RIGHT,RIGHT,DOWN,RIGHT,DOWN,DOWN,RIGHT,DOWN
UP,DOWN,RIGHT,UP,LEFT,RIGHT,DOWN,LEFT
RIGHT,DOWN,RIGHT,RIGHT,DOWN,UP,LEFT,DOWN
DOWN,LEFT,UP,DOWN,RIGHT,UP,LEFT,RIGHT
DOWN,UP,LEFT,DOWN,RIGHT,RIGHT,DOWN,RIGHT
